<img src="https://www.ost.ch/typo3conf/ext/template/Resources/Public/Images/logo-de.svg" width="300" height="250" align="right"/>

# Modeling

von <a href="mailto:pascal.schaback@ost.ch"> Pascal Schaback</a>  

Referent: <a href="mailto:christoph.wuersch@ost.ch"> Christoph Würsch </a>  
Co-Referent: <a href="mailto:klaus.frick@ost.ch"> Klaus Frick </a>

### Inhalt
- Packages
- Datengrundlage
- Datenaufbereitung
    - Glätten
- Trainieren der Modelle
    - Training-, Testdaten
    - Skalierung & Normierung
    - Metriken
    - Modelle
        - Dummy
        - Naives Modell
        - lin Regression
            - Ridge
            - Lasso
        - k Nächste Nachbarn
        - Entscheidungsbäume
        - Zufallswald
        - Neuronale Netze
        - Voting
- Modellvergleich

## Packages

In [32]:
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
PROJECT_DIR = str(Path(globals()['_dh'][0])).rsplit("\\",1)[0] + "\\"

In [33]:
# CORE
import warnings

# OTHER
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.offline import plot
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.model_selection import TimeSeriesSplit, cross_validate
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler, RobustScaler, KBinsDiscretizer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.base import clone
from sklearn.pipeline import make_pipeline

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

from tqdm import tqdm
from timeit import default_timer as timer

# OWN
from src.caching import check_cache, caching, save_to_cache
from src import processing
from src import meteomatics_api

## Settings

In [34]:
# Design Vorlage definieren
px.defaults.template = "seaborn"
px.defaults.color_continuous_scale = "Turbo"

# Interaktive Grafiken in HTML
import plotly.io as pio
pio.renderers.default='notebook'

In [35]:
export = PROJECT_DIR + "exports/models/publish/"
save_to_folder = True
plot_in_browser = False
plot_in_notebook = False

-------
## Daten laden

In [36]:
df = pd.read_csv(PROJECT_DIR + "src/data/datalake_Quinten_2021-23_hPa-kmh-C-W_50km.zip", index_col=0)
df.index = pd.to_datetime(df.index, unit='ns')
y = df[[column for column in df.columns if column.startswith("Quinten, wind_gusts")]]
y.head()

,"Quinten, wind_gusts_10m:kmh_x","Quinten, wind_gusts_10m:kmh_y"
validdate,,
2021-01-01 00:20:00+00:00,34.194791,0.596872
2021-01-01 00:30:00+00:00,39.194030,-0.684134
2021-01-01 00:40:00+00:00,39.304828,-4.826026
2021-01-01 00:50:00+00:00,36.958823,-3.233478
2021-01-01 01:00:00+00:00,31.922050,-2.232207


In [37]:
X = df.copy()
X.head()

,"Bad Ragaz, diff_global_rad:W","Bad Ragaz, diff_msl_pressure:hPa","Bad Ragaz, diff_t_2m:C","Bad Ragaz, diff_wind_gusts_10m:kmh_x","Bad Ragaz, diff_wind_gusts_10m:kmh_y","Bad Ragaz, diff_wind_speed_10m:kmh_x","Bad Ragaz, diff_wind_speed_10m:kmh_y","Bad Ragaz, global_rad:W","Bad Ragaz, msl_pressure:hPa","Bad Ragaz, t_2m:C",...,"Wädenswil, wind_speed_10m:kmh_x","Wädenswil, wind_speed_10m:kmh_y","bise, diff_msl_pressure:hPa","bise, msl_pressure:hPa","föhn, diff_msl_pressure:hPa","föhn, msl_pressure:hPa","time, cos_day","time, sin_day","time, cos_year","time, sin_year"
validdate,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:20:00+00:00,0.0,0.10,-0.30,-0.801906,0.408591,0.356403,-0.181596,0.0,1006.8,2.6,...,-5.768227,0.606265,0.05,-0.2,-0.10,4.7,0.996141,0.087764,1.0,0.000239
2021-01-01 00:30:00+00:00,0.0,-0.05,0.05,-0.624472,-0.844828,-0.604331,-0.338196,0.0,1006.6,3.0,...,-7.190133,-0.376819,0.05,-0.3,-0.30,4.4,0.991325,0.131434,1.0,0.000359
2021-01-01 00:40:00+00:00,0.0,-0.25,0.35,2.372627,-2.033414,0.111114,-0.536431,0.0,1006.3,3.3,...,-8.254532,-0.867586,-0.10,-0.4,-0.10,4.5,0.984595,0.174850,1.0,0.000478
2021-01-01 00:50:00+00:00,0.0,-0.30,0.30,0.461712,-6.143016,2.543926,-5.247327,0.0,1006.0,3.6,...,-12.980574,-5.244492,0.05,-0.2,0.05,4.5,0.975965,0.217929,1.0,0.000598
2021-01-01 01:00:00+00:00,0.0,-0.10,0.10,-0.486392,-5.376159,1.248055,-4.459618,0.0,1006.1,3.5,...,-12.830494,-6.537463,0.10,-0.2,0.05,4.6,0.965450,0.260587,1.0,0.000717


## Datenverarbeitung

### Glätten
Minimales Glätten der Daten...

In [38]:
def plot_timeplots(df, comment, cols=4):
    df_stack = processing.df_to_stack_columns(df)
    
    unique_params = list(set(column for column in df_stack.columns.get_level_values(1)))
    print("unterschiedliche Parameter Anzahl:", len(unique_params))
    print(unique_params)

    titles = []
    figs = []
    for location, param in df_stack.columns:
        if param in unique_params:
            fig = px.line(df_stack.iloc[:6*24*31,:].loc[:,location][param])
            figs.append(fig)   # reduce amount of data to 1 week
            unique_params.remove(param)
            titles.append(param)
            
    rows = int(np.ceil(len(titles)/cols))
    fig = make_subplots(rows=rows, cols=cols, subplot_titles=titles)
    title = "Zeitsignale der verschiedenen Messgrössen an zufälligen Stationen"
    subtitle = "<br><sub>"+ comment + "</sub>"
    unique_params = titles        

    for i, figure in enumerate(figs):
        for trace in range(len(figure["data"])):
            fig.append_trace(figure["data"][trace], row=int(np.ceil((i+1)/cols)), col=i%cols+1)

    fig.update_layout(title_text=title + subtitle, showlegend=False)

    # viewing the plot in browser:
    if plot_in_browser:
        plot(fig)
    if plot_in_notebook:
        fig.show()

    # save plot to export
    if save_to_folder:
        fig.write_html(export + title + "_" + comment + ".html")
        
    return fig

In [39]:
_ = plot_timeplots(X, comment="vor dem glätten")

unterschiedliche Parameter Anzahl: 18
['sin_year', 'cos_year', 't_2m:C', 'diff_wind_speed_10m:kmh_y', 'diff_global_rad:W', 'diff_wind_gusts_10m:kmh_x', 'diff_t_2m:C', 'diff_wind_speed_10m:kmh_x', 'diff_msl_pressure:hPa', 'diff_wind_gusts_10m:kmh_y', 'global_rad:W', 'wind_gusts_10m:kmh_y', 'wind_speed_10m:kmh_y', 'sin_day', 'cos_day', 'wind_speed_10m:kmh_x', 'wind_gusts_10m:kmh_x', 'msl_pressure:hPa']


In [40]:
X = X.rolling(3).mean()
X.dropna(inplace=True)
X.head()

,"Bad Ragaz, diff_global_rad:W","Bad Ragaz, diff_msl_pressure:hPa","Bad Ragaz, diff_t_2m:C","Bad Ragaz, diff_wind_gusts_10m:kmh_x","Bad Ragaz, diff_wind_gusts_10m:kmh_y","Bad Ragaz, diff_wind_speed_10m:kmh_x","Bad Ragaz, diff_wind_speed_10m:kmh_y","Bad Ragaz, global_rad:W","Bad Ragaz, msl_pressure:hPa","Bad Ragaz, t_2m:C",...,"Wädenswil, wind_speed_10m:kmh_x","Wädenswil, wind_speed_10m:kmh_y","bise, diff_msl_pressure:hPa","bise, msl_pressure:hPa","föhn, diff_msl_pressure:hPa","föhn, msl_pressure:hPa","time, cos_day","time, sin_day","time, cos_year","time, sin_year"
validdate,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:40:00+00:00,0.0,-0.066667,3.333333e-02,0.315416,-0.823217,-0.045605,-0.352074,0.0,1006.566667,2.966667,...,-7.070964,-0.212713,3.786786e-14,-0.300000,-1.666667e-01,4.533333,0.990687,0.131349,1.0,0.000359
2021-01-01 00:50:00+00:00,0.0,-0.200000,2.333333e-01,0.736622,-3.007086,0.683570,-2.040651,0.0,1006.300000,3.300000,...,-9.475079,-2.162966,3.786786e-14,-0.300000,-1.166667e-01,4.466667,0.983962,0.174738,1.0,0.000478
2021-01-01 01:00:00+00:00,0.0,-0.216667,2.500000e-01,0.782649,-4.517530,1.301032,-3.414458,0.0,1006.133333,3.466667,...,-11.355200,-4.216514,1.666667e-02,-0.266667,-3.238150e-17,4.533333,0.975337,0.217789,1.0,0.000598
2021-01-01 01:10:00+00:00,0.0,-0.100000,-3.700743e-17,0.082418,-2.917410,0.943195,-2.361528,0.0,1006.100000,3.300000,...,-14.157630,-6.757193,3.333333e-02,-0.233333,1.666667e-02,4.500000,0.964829,0.260419,1.0,0.000717
2021-01-01 01:20:00+00:00,0.0,0.016667,-1.000000e-01,0.569094,-0.660972,-0.058712,-0.034953,0.0,1006.166667,3.266667,...,-13.744568,-7.550677,-6.666667e-02,-0.400000,3.333333e-02,4.600000,0.952459,0.302547,1.0,0.000837


In [41]:
_ = plot_timeplots(X, comment="nach glätten")

unterschiedliche Parameter Anzahl: 18
['sin_year', 'cos_year', 't_2m:C', 'diff_wind_speed_10m:kmh_y', 'diff_global_rad:W', 'diff_wind_gusts_10m:kmh_x', 'diff_t_2m:C', 'diff_wind_speed_10m:kmh_x', 'diff_msl_pressure:hPa', 'diff_wind_gusts_10m:kmh_y', 'global_rad:W', 'wind_gusts_10m:kmh_y', 'wind_speed_10m:kmh_y', 'sin_day', 'cos_day', 'wind_speed_10m:kmh_x', 'wind_gusts_10m:kmh_x', 'msl_pressure:hPa']


### Skalierung

In [42]:
import plotly.figure_factory as ff

def plot_distplots(df, comment, cols=4):
    df_stack = processing.df_to_stack_columns(df)
    
    unique_params = list(set(column for column in df_stack.columns.get_level_values(1)))
    print("unterschiedliche Parameter Anzahl:", len(unique_params))
    print(unique_params)

    titles = []
    figs = []
    for location, param in df_stack.columns:
        if param in unique_params:
            fig = ff.create_distplot([df_stack.loc[:,location][param].sample(10000).values], group_labels=[location+param])
            figs.append(fig)   # reduce amount of data to 1 week
            unique_params.remove(param)
            titles.append(param)
            
    rows = int(np.ceil(len(titles)/cols))
    fig = make_subplots(rows=rows, cols=cols, subplot_titles=titles)
    title = "Histogramme der verschiedenen Messgrössen an zufälligen Stationen"
    subtitle = "<br><sub>"+ comment + "</sub>"
    unique_params = titles        

    for i, figure in enumerate(figs):
        for trace in range(len(figure["data"])):
            fig.append_trace(figure["data"][trace], row=int(np.ceil((i+1)/cols)), col=i%cols+1)

    fig.update_layout(title_text=title + subtitle, showlegend=False)

    # viewing the plot in browser:
    if plot_in_browser:
        plot(fig)
    if plot_in_notebook:
        fig.show()

    # save plot to export
    if save_to_folder:
        fig.write_html(export + title + "_" + comment + ".html")
        
    return fig

In [43]:
_ = plot_distplots(X, comment="vor dem Skalieren")

unterschiedliche Parameter Anzahl: 18
['sin_year', 'cos_year', 't_2m:C', 'diff_wind_speed_10m:kmh_y', 'diff_global_rad:W', 'diff_wind_gusts_10m:kmh_x', 'diff_t_2m:C', 'diff_wind_speed_10m:kmh_x', 'diff_msl_pressure:hPa', 'diff_wind_gusts_10m:kmh_y', 'global_rad:W', 'wind_gusts_10m:kmh_y', 'wind_speed_10m:kmh_y', 'sin_day', 'cos_day', 'wind_speed_10m:kmh_x', 'wind_gusts_10m:kmh_x', 'msl_pressure:hPa']


In [44]:
kbin = KBinsDiscretizer(n_bins=9, encode="ordinal", strategy="uniform")

ct_preprocess = ColumnTransformer([
    # measurements
    ("temp",    make_pipeline(StandardScaler()), make_column_selector(pattern="t_")),
    ("wind",    make_pipeline(PowerTransformer()), make_column_selector(pattern="wind_gusts")),
    ("pres",    make_pipeline(StandardScaler()), make_column_selector(pattern="msl_")),
    ("radi",     make_pipeline(PowerTransformer()), make_column_selector(pattern=" global_")),
    ("dradi",     make_pipeline(MinMaxScaler()), make_column_selector(pattern="diff_global_")),
    # time
    ("time",    make_pipeline(StandardScaler()), make_column_selector(pattern="time,")),
    ],
    remainder="drop"
).set_output(transform="pandas")
ct_preprocess

ColumnTransformer(transformers=[('temp',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000249DEE53430>),
                                ('wind',
                                 Pipeline(steps=[('powertransformer',
                                                  PowerTransformer())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000249DEE53520>),
                                ('pres',
                                 Pipeline(s...
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000249DF8556A0>),
                                ('dradi',
                                 Pipeline(steps=[('minmaxscaler',
                                                  MinMaxScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000249DF855AF0>),
                                ('time',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000249DF855DC0>)])

### Transformer testen

In [45]:
test = ct_preprocess.fit_transform(X)

In [46]:
_ = plot_distplots(test, comment="nach Column Transformer")

unterschiedliche Parameter Anzahl: 14
['cos_year', 't_2m:C', 'diff_global_rad:W', 'diff_wind_gusts_10m:kmh_x', 'diff_t_2m:C', 'global_rad:W', 'diff_msl_pressure:hPa', 'diff_wind_gusts_10m:kmh_y', 'sin_day', 'wind_gusts_10m:kmh_y', 'sin_year', 'cos_day', 'wind_gusts_10m:kmh_x', 'msl_pressure:hPa']


------
## X, Y definieren

### Y - Unterschiedliche Zeiten hinzufügen

In [47]:
y_columns = [column for column in y.columns if "wind_gusts" in column]
print(y_columns)

['Quinten, wind_gusts_10m:kmh_x', 'Quinten, wind_gusts_10m:kmh_y']


In [48]:
Y = pd.DataFrame()
for y_column in y_columns:
    for timeshift in range(1,19):
        Y["_+".join([y_column,str(timeshift)])+"0min"] = y[y_column].shift(-timeshift)
Y = Y.dropna()

### Match index

In [49]:
idxs = Y.index.join(X.index, how="inner")
X = X.loc[idxs]
Y = Y.loc[idxs]
print(X.shape)
print(Y.shape)

(105099, 264)
(105099, 36)


In [50]:
X

,"Bad Ragaz, diff_global_rad:W","Bad Ragaz, diff_msl_pressure:hPa","Bad Ragaz, diff_t_2m:C","Bad Ragaz, diff_wind_gusts_10m:kmh_x","Bad Ragaz, diff_wind_gusts_10m:kmh_y","Bad Ragaz, diff_wind_speed_10m:kmh_x","Bad Ragaz, diff_wind_speed_10m:kmh_y","Bad Ragaz, global_rad:W","Bad Ragaz, msl_pressure:hPa","Bad Ragaz, t_2m:C",...,"Wädenswil, wind_speed_10m:kmh_x","Wädenswil, wind_speed_10m:kmh_y","bise, diff_msl_pressure:hPa","bise, msl_pressure:hPa","föhn, diff_msl_pressure:hPa","föhn, msl_pressure:hPa","time, cos_day","time, sin_day","time, cos_year","time, sin_year"
validdate,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:40:00+00:00,0.0,-0.066667,3.333333e-02,0.315416,-0.823217,-0.045605,-0.352074,0.0,1006.566667,2.966667,...,-7.070964,-0.212713,3.786786e-14,-0.300000,-1.666667e-01,4.533333,0.990687,0.131349,1.000000,0.000359
2021-01-01 00:50:00+00:00,0.0,-0.200000,2.333333e-01,0.736622,-3.007086,0.683570,-2.040651,0.0,1006.300000,3.300000,...,-9.475079,-2.162966,3.786786e-14,-0.300000,-1.166667e-01,4.466667,0.983962,0.174738,1.000000,0.000478
2021-01-01 01:00:00+00:00,0.0,-0.216667,2.500000e-01,0.782649,-4.517530,1.301032,-3.414458,0.0,1006.133333,3.466667,...,-11.355200,-4.216514,1.666667e-02,-0.266667,-3.238150e-17,4.533333,0.975337,0.217789,1.000000,0.000598
2021-01-01 01:10:00+00:00,0.0,-0.100000,-3.700743e-17,0.082418,-2.917410,0.943195,-2.361528,0.0,1006.100000,3.300000,...,-14.157630,-6.757193,3.333333e-02,-0.233333,1.666667e-02,4.500000,0.964829,0.260419,1.000000,0.000717
2021-01-01 01:20:00+00:00,0.0,0.016667,-1.000000e-01,0.569094,-0.660972,-0.058712,-0.034953,0.0,1006.166667,3.266667,...,-13.744568,-7.550677,-6.666667e-02,-0.400000,3.333333e-02,4.600000,0.952459,0.302547,1.000000,0.000837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 20:20:00+00:00,0.0,0.083333,-1.000000e-01,-1.424251,1.379831,-0.815936,0.786323,0.0,1023.033333,15.233333,...,-1.488976,-1.673331,8.333333e-02,1.533333,6.666667e-02,11.166667,0.567699,-0.822454,0.999996,-0.002749
2022-12-31 20:30:00+00:00,0.0,0.016667,-1.665335e-15,-0.641084,0.907199,0.119956,0.108442,0.0,1023.033333,15.233333,...,-0.731499,-3.484822,1.500000e-01,1.700000,1.333333e-01,11.333333,0.603277,-0.796725,0.999997,-0.002630
2022-12-31 20:40:00+00:00,0.0,-0.066667,3.333333e-02,1.580078,0.537182,1.531336,-0.159791,0.0,1022.900000,15.300000,...,-0.306666,-4.515388,1.833333e-01,1.900000,8.333333e-02,11.333333,0.637690,-0.769457,0.999997,-0.002510


In [51]:
X[X.isna().any(axis=1)]

,"Bad Ragaz, diff_global_rad:W","Bad Ragaz, diff_msl_pressure:hPa","Bad Ragaz, diff_t_2m:C","Bad Ragaz, diff_wind_gusts_10m:kmh_x","Bad Ragaz, diff_wind_gusts_10m:kmh_y","Bad Ragaz, diff_wind_speed_10m:kmh_x","Bad Ragaz, diff_wind_speed_10m:kmh_y","Bad Ragaz, global_rad:W","Bad Ragaz, msl_pressure:hPa","Bad Ragaz, t_2m:C",...,"Wädenswil, wind_speed_10m:kmh_x","Wädenswil, wind_speed_10m:kmh_y","bise, diff_msl_pressure:hPa","bise, msl_pressure:hPa","föhn, diff_msl_pressure:hPa","föhn, msl_pressure:hPa","time, cos_day","time, sin_day","time, cos_year","time, sin_year"
validdate,,,,,,,,,,,,,,,,,,,,,


In [52]:
Y[Y.isna().any(axis=1)]

,"Quinten, wind_gusts_10m:kmh_x_+10min","Quinten, wind_gusts_10m:kmh_x_+20min","Quinten, wind_gusts_10m:kmh_x_+30min","Quinten, wind_gusts_10m:kmh_x_+40min","Quinten, wind_gusts_10m:kmh_x_+50min","Quinten, wind_gusts_10m:kmh_x_+60min","Quinten, wind_gusts_10m:kmh_x_+70min","Quinten, wind_gusts_10m:kmh_x_+80min","Quinten, wind_gusts_10m:kmh_x_+90min","Quinten, wind_gusts_10m:kmh_x_+100min",...,"Quinten, wind_gusts_10m:kmh_y_+90min","Quinten, wind_gusts_10m:kmh_y_+100min","Quinten, wind_gusts_10m:kmh_y_+110min","Quinten, wind_gusts_10m:kmh_y_+120min","Quinten, wind_gusts_10m:kmh_y_+130min","Quinten, wind_gusts_10m:kmh_y_+140min","Quinten, wind_gusts_10m:kmh_y_+150min","Quinten, wind_gusts_10m:kmh_y_+160min","Quinten, wind_gusts_10m:kmh_y_+170min","Quinten, wind_gusts_10m:kmh_y_+180min"
validdate,,,,,,,,,,,,,,,,,,,,,


--------
## Sklearn

### Testdatensatz
Letztes halbes Jahr als Testdaten

In [53]:
X_test = X.iloc[int(3*X.shape[0]/4):,:]
X = X.drop(index=X_test.index)
print(X.shape)
print(X_test.shape)

(78824, 264)
(26275, 264)


In [54]:
print("Train-Validation:\t", X.index[0]," - ", X.index[-1])
print("Testing:\t\t", X_test.index[0]," - ", X_test.index[-1])

Train-Validation:	 2021-01-01 00:40:00+00:00  -  2022-07-02 09:50:00+00:00
Testing:		 2022-07-02 10:00:00+00:00  -  2022-12-31 21:00:00+00:00


In [55]:
Y_test = Y.loc[X_test.index]
Y = Y.drop(index=Y_test.index)
print(Y.shape)
print(Y_test.shape)

(78824, 36)
(26275, 36)


#### Skaliere X

In [56]:
X = ct_preprocess.fit_transform(X)
X_test = ct_preprocess.transform(X_test)

speichern für prediktion

In [57]:
save_to_cache(ct_preprocess, "ColumnTransformer", folder="models")

save to cache


True

### Trainings-, Validierungssplit

#### Zeitreihen Split (skip)

In [918]:
ts_split = TimeSeriesSplit(
    n_splits=3,                             # set low due to performance issues
    gap=0,                                  # if over more years use here one year gap
    #max_train_size=6*24*180,               # max 1/2 year of train size data
    #test_size=6*24*31*2,                      # test data set of 2 months
)

#### Stratified Split
Alternativer Split, um selten vorkommende Ereignisse stärker zu gewichten.  
Verwenden eines Validierungs Datensatztes

In [919]:
def get_y_groups(y, y_column, n_bins=7):
    strat_bin = KBinsDiscretizer(n_bins=n_bins, encode="ordinal", strategy="kmeans").set_output(transform="pandas")
    if isinstance(y_column,str):
        groups = strat_bin.fit_transform(y[y_column].array.reshape(-1, 1))
    else:
        groups = strat_bin.fit_transform(y[y_column])
    return groups.set_axis([y_column], axis=1)

In [920]:
idx = get_y_groups(y,y_columns[0])
idx.columns
idx.reset_index().groupby(y_columns[0]).count()

c:\Users\SCPA\GIT\Surfalarm\weatherdata\venv\lib\site-packages\sklearn\preprocessing\_discretization.py:239: FutureWarning:

In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.



,index
"Quinten, wind_gusts_10m:kmh_x",
0.0,3919
1.0,10267
2.0,13854
3.0,39259
4.0,26310
5.0,8511
6.0,2999


In [921]:
fig = px.scatter(y[y_columns[0]].iloc[idx.index], color=idx[y_columns[0]],title="Gruppen auf Basis der zu prognostizierenden Grösse",
           labels={"value":y_columns[0],
                   "color":"group"})
plot(fig)

'temp-plot.html'

In [922]:
from sklearn.model_selection import StratifiedShuffleSplit

sss_split = StratifiedShuffleSplit(n_splits=3, test_size=0.1)

#### Visualisierung der Trainings und Testdatensätze

In [923]:
def show_train_and_test(y, X, splitter, y_columns, comment="", vectors=True):
    ts_df = pd.DataFrame()
    figs = []
    titles = []
    i=1
    groups = get_y_groups(y.loc[X.index], y_columns)
    for train, test in list(splitter.split(X, groups)):
        train = pd.DataFrame(y.iloc[train])
        train["use"] = "train"
        test = pd.DataFrame(y.iloc[test])
        test["use"] = "val"
        tt_set = pd.concat([train,test])
        tt_set["split"] = i
        ts_df = pd.concat([ts_df, tt_set])
        titles.append("Datasplit "+str(i))
        if vectors:
            fig = px.scatter(tt_set, x=y_columns[0], y=y_columns[1], color="use", opacity=0.8)
        else:
            fig = px.scatter(tt_set, y=y_columns[0], color="use")
        figs.append(fig)
        i+=1
    
    fig = make_subplots(rows=3, cols=3, subplot_titles=titles)
    
    for i, figure in enumerate(figs):
        for trace in range(len(figure["data"])):
            fig.append_trace(figure["data"][trace], row=int(np.ceil((i+1)/3)), col=i%3+1)

    title = "Trainings-Validierungs Split "+ comment
    #subtitle = "<br><sub>" + ", ".join(column) + "</sub>"
    fig.update_layout(title_text=title, showlegend=True)
    fig.layout.yaxis.scaleanchor="x"
    fig.update_traces(marker={'size': 3})
    
    # viewing the plot in browser:
    if plot_in_browser:
        plot(fig)
    else:
        fig.show()

    # save plot to export
    if save_to_folder:
        fig.write_html(export + "train_val_splits_"+ comment + ".html")
        
    return fig, ts_df

In [ ]:
fig, ts_df = show_train_and_test(y, X, ts_split, y_columns, comment="with Time Split Vectors", vectors=True)
fig, ts_df = show_train_and_test(y, X, ts_split, y_columns, comment="with Time Split", vectors=False)

use kmeans to make classes for stratified split

In [ ]:
fig, ts_df = show_train_and_test(y, X, sss_split, y_columns, comment="with Stratified Kmeans Split Vectors", vectors=True)
fig, ts_df = show_train_and_test(y, X, sss_split, y_columns, comment="with Stratified Kmeans Split", vectors=False)

### Metriken

#### Scores

In [926]:
def calculate_scores(y_hat, y_true):
    scores = {
            "mean_absolute_error"       : mean_absolute_error(y_true, y_hat),
            "root_mean_squared_error"   : mean_squared_error(y_true, y_hat, squared=False),
            #"mean_absolute_percentage_error" : mean_absolute_percentage_error(y_true, y_hat),
            #"my_loss"                   : loss(y_true, y_hat),
            "r2"                        : r2_score(y_true, y_hat),
        }
    return scores

In [927]:
def evaluate(X, Y, model, splitter, every=1, y_groups=None, random_state=None, **kwargs):
    
    print("Only every %i column will be taken to fit" %(every))
    
    cv_dfs = pd.DataFrame()
    
    criterions = {}
    for criterion, Y in Y.items():
        print("building models for:",criterion)
        models = {}
        predictions = {}
        for column in tqdm(Y.columns[::every]):
            columnname, time = column.split("_+")
            y_true = Y[column]
            if "important" in kwargs: #use X[important]
                X_ = X.loc[:, kwargs["important"][column]]
            sub_models = {}
            prediction = {}
            cv_df = pd.DataFrame()
            i = 1
            for train, test in tqdm(list(splitter.split(X, y_groups))):
                
                if "custom_y_hat" in kwargs: # no model generated and needed and no predict needed
                    sub_model = None
                    time_fit = float("nan")
                    time_predict = float("nan")
                    y_hat = pd.DataFrame({criterion : kwargs["custom_y_hat"][criterion].iloc[test]})
                    
                else: #train model and predict values
                    time_start = timer()
                    if "important" in kwargs:
                        sub_model = clone(model).fit(X_.iloc[train], y_true.iloc[train])
                    else:
                        sub_model = clone(model).fit(X.iloc[train], y_true.iloc[train])
                    time_fit = timer() - time_start
                    
                    #predict
                    if "important" in kwargs:
                        y_hat = sub_model.predict(X_.iloc[test])
                    else:
                        y_hat = sub_model.predict(X.iloc[test])
                    time_predict = (timer() - time_start) - time_fit
                    
                    #create dataframe with time information
                    y_hat = pd.DataFrame({criterion : y_hat}, 
                                        index=Y.iloc[test].index + pd.Timedelta(time))
            
                #evaluate
                scores = pd.DataFrame(calculate_scores(y_hat, y_true.iloc[test]), index=[i])
                
                #append other important information
                scores["training time"] = time_fit
                scores["predicting time"] = time_predict
                scores["train_size"] = train.shape[0]
                scores["criterion"] = criterion
                
                cv_df = pd.concat([cv_df, scores])
                
                sub_models[i] = sub_model
                prediction[i] = y_hat
                i+=1
            
            cv_df["criterion"] = columnname
            cv_df["forcast time"] = time
            cv_df.reset_index(names="fold", inplace=True)
            cv_dfs = pd.concat([cv_dfs, cv_df])
            models[time] = sub_models
            predictions[time] = prediction
            
        criterions[criterion] = {
            "submodels" : models,
            "predictions" : predictions,
        }
        
    cv_dfs["source"]="training"
    
    results = {
        "criterions" : criterions,
        "scores" : cv_dfs
    }
    return results 

In [928]:
def evaluate_on_testing(criterions, X, X_test=X_test, Y_test=Y_test, **kwargs):
    
    forcasts = pd.DataFrame()
    test_scores = pd.DataFrame()
    print("evaluate on testing...")
    for criterion in tqdm(criterions.keys()):
        first = True
        for time, submodels in criterions[criterion]["submodels"].items():
            column = "_+".join([criterion, time])
            y_true = pd.DataFrame(Y_test[column])
            
            X_test = X_test[[col for col in X.columns]]
            
            if "important" in kwargs:
                X_test_ = X_test.loc[:, kwargs["important"][column]]
            
            time_start = timer()
            
            if "custom_y_hat" in kwargs:
                if "important" in kwargs:
                    y_hats = df.loc[X_test_.index, criterion]
                else:
                    y_hats = df.loc[X_test.index, criterion]
                y_hats.index += pd.Timedelta("+10min")
                y_hats = pd.DataFrame(y_hats)
                fold=None
            else:              
                y_hats = pd.DataFrame()
                for fold, sub_model in submodels.items():
                    if "important" in kwargs:
                        y_hat = sub_model.predict(X_test_)
                    else:
                        y_hat = sub_model.predict(X_test)
                    y_hats["fold" + str(fold)] = y_hat
            time_predict = (timer() - time_start)
        
            #evaluate
            scores = pd.DataFrame()
            for fold in y_hats.columns:
                score = pd.DataFrame(calculate_scores(y_hats[fold], y_true), index=[time])
                score["fold"] = fold
                scores = pd.concat([scores, score])
 
            #append other important information
            scores["training time"] = None
            scores["predicting time"] = time_predict
            scores["criterion"] = criterion
            scores["forcast time"] = time
            scores["submodels"] = fold
            
            test_scores = pd.concat([test_scores, scores])
            
            #add predictions
            y_hats = pd.DataFrame({criterion: y_hats.mean(axis=1).values}, index=y_true.index + pd.Timedelta(time))
            y_hats["use"] = "forcast +" + time
        
            if first:# only once!
                y_true = y_true.set_axis([criterion], axis=1)
                y_true.index += pd.Timedelta(time)
                y_true["use"]       = "ground truth"
                forcasts = pd.concat([forcasts, y_true])
                first=False
            forcasts = pd.concat([forcasts, y_hats])
    
    test_scores["source"]="testing"
    
    results = {
        "forcasts" : forcasts,
        "test_scores" : test_scores
    }
    return results

In [929]:
def print_scores(scores, test_scores, **kwargs):
    return pd.concat([scores, test_scores]).drop(columns=["submodels","fold"]).groupby(["source","criterion"]).describe().loc[:,(slice(None), slice("mean", "std"))].T

### Visualisierungen

#### Scores
using calculated scores from evaluate

In [930]:
def plot_score_of_model(scores, test_scores, model_name, color="train_size", **kwargs):
    scores = pd.concat([scores, test_scores]).drop(columns=["submodels","fold","train_size"]).melt(id_vars=["source", "criterion", "forcast time"], value_name="score_value", var_name="score_name")
    
    title = "Metriken <br><sub>" + model_name +"<sub>"
    fig = px.box(scores, facet_col="score_name", facet_row="criterion", x="forcast time", y="score_value", color="source", title = title)
    
    fig.update_traces(marker={'size': 7})
    fig.update_yaxes(matches=None)

    fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
    
    #add competitiors
    fig.add_hline(y=8.66, line_dash="solid", row=0, col=1, line_color="red", line_width=3,
        annotation_text="MF-AROME", 
        annotation_position="top left")
    fig.add_hline(y=12.01, line_dash="solid", row=0, col=2, line_color="red", line_width=3,
        annotation_text="MF-AROME", 
        annotation_position="top left")
    fig.add_hline(y=0.14, line_dash="solid", row=0, col=3, line_color="red", line_width=3,
         annotation_text="MF-AROME", 
         annotation_position="bottom left")
    
    fig.add_hline(y=4.96, line_dash="solid", row=1, col=1, line_color="red", line_width=3,
        annotation_text="MM-SWISS1K", 
        annotation_position="top left")
    fig.add_hline(y=6.67, line_dash="solid", row=1, col=2, line_color="red", line_width=3,
        annotation_text="MM-SWISS1K", 
        annotation_position="top left")
    fig.add_hline(y=-1.29, line_dash="solid", row=1, col=3, line_color="red", line_width=3,
         annotation_text="MM-SWISS1K", 
         annotation_position="bottom left")
    
    # viewing the plot in browser:
    if plot_in_browser:
        plot(fig)
    if plot_in_notebook:
        fig.show()

    # save plot to export
    if save_to_folder:
        fig.write_html(export + "Metriken_" + model_name + ".html")
        
    return fig

#### Forcasts
Using pretrained models from evaluate

In [931]:
def plot_forcasts(forcasts, model_name, starttime="2022-08-01", endtime="2022-08-31", **kwargs):
    
    forcasts_melted = forcasts.copy().melt(ignore_index=False, id_vars=["use"],var_name="criterion", value_name=y_columns[0].replace("_x",""))
    forcast_melted = forcasts_melted.loc[starttime:endtime]
    title = "Vorhersage mit "+ model_name+ " auf den Testdaten"
    fig = px.line(forcast_melted, 
                  y=y_columns[0].replace("_x",""), 
                  color="use", 
                  facet_row="criterion",
                  title=title + "<br><sub>Für den Forcast wurden aus allen Modellen der Mittelwert berechnet"
                  )
    #                        +"<br><sup>"+y_column+"</sup>")

    fig.update_traces(line=dict(width=1))
    fig.update_yaxes(matches=None)
    
    fig.for_each_trace(
        lambda trace: trace.update(visible="legendonly") if (trace.name.startswith("forcast") and not "+100" in trace.name) else (),
    )

    # viewing the plot in browser:
    if plot_in_browser:
        plot(fig)
    if plot_in_notebook:
        fig.show()
    
    # save plot to export
    if save_to_folder:
        fig.write_html(export + title +".html")
    
    return fig

#### Feature Importance

In [932]:
def plot_feature_importance(X, criterions, model_name, **kwargs):

    feature_importances = pd.DataFrame()
    
    for criterion in criterions.keys():
        for time, models in criterions[criterion]["submodels"].items():
            f_is = pd.DataFrame()
            for fold, model in models.items():
                if "Pipeline" in str(type(model)):
                    try:
                        coefs = np.abs(model[-1].coef_)
                    except:
                        coefs = np.abs(model[-1].feature_importances_)
                    try:
                        if coefs.shape[0] < coefs.shape[1]:
                            coefs = coefs.T
                        idxs = ["PC_%i" %(coef) for coef in range(len(coefs))]
                    except:
                        idxs = ["PC_%i" %(coef) for coef in range(len(coefs))]
                else:
                    try:
                        coefs = np.abs(model.coef_)
                    except:
                        coefs = np.abs(model.feature_importances_)
                    try:
                        if coefs.shape[0] < coefs.shape[1]:
                            coefs = coefs.T
                        idxs = list(X.columns)
                    except:
                        idxs = list(X.columns)
                    
                f_i = pd.DataFrame(coefs,
                                index = idxs,
                                columns = ["feature_importance"]
                                )
                
                f_i["time [min]"] = int(time.replace("min",""))
                f_i["fold"]       = fold
                f_is = pd.concat([f_is, f_i])
                
            f_is["criterion"]  = criterion
            feature_importances = pd.concat([feature_importances, f_is])
            
    titles = []
    figs = []   
    for criterion in feature_importances["criterion"].unique():
        f_i_set = feature_importances[feature_importances["criterion"]==criterion].drop(columns="criterion").reset_index()
        fig = px.box(f_i_set.sort_values(["time [min]","feature_importance"]),
                    x="index", y="feature_importance", color="time [min]")
        fig.update_layout(showlegend=False)
        fig.update_xaxes(categoryorder='array', 
                        categoryarray=list(f_i_set.groupby("index").mean().sort_values("feature_importance", ascending=False).index))
        figs.append(fig)
        titles.append(criterion)
        
    fig = make_subplots(cols=len(feature_importances["criterion"].unique()), rows=1, subplot_titles=titles)
    
    for i, figure in enumerate(figs):
        for trace in range(len(figure["data"])):
            fig.append_trace(figure["data"][trace], col=int(np.ceil((i+1)/1)), row=i%1+1)
            
    title = "Wichtigste Merkmale für " + model_name
    fig.update_layout(title_text=title)
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.2,
        xanchor="right",
        x=1
    ))
    
    # viewing the plot in browser:
    if plot_in_browser:
        plot(fig)
    if plot_in_notebook:
        fig.show()
            
    # save plot to export
    if save_to_folder:
        fig.write_html(export + title + ".html")
    return fig

### Modelle

Um auf alle trainierten Modelle auch im späteren Verlauf zugreifen zu können.

In [933]:
all_models = {}

In [934]:
def save_scores(all_models, model_name, scores, test_scores, forcasts, **kwargs):
    all_models[model_name] = {
        "scores"    : scores,
        "test_scores": test_scores,
        "forcasts"  : forcasts
    }
    return all_models

In [935]:
def evaluate_model(model, all_models=all_models, plot=False, **kwargs):
    print(model["model_name"])
    print("building models...")
    
    if "Stratified" in str(type(model["splitter"])).split(".")[-1]:
        print("stratified split identified. building groups from y...")
        model["y_groups"] = get_y_groups(y.loc[model["X"].index], y_columns)  
    if "important" in kwargs:
        print("important features will be taken from dict...")
    model.update(evaluate(**model))
    
    model.update(evaluate_on_testing(**model))
    all_models = save_scores(all_models, **model)
    
    if plot:
        _ = plot_score_of_model(**model)
        _ = plot_forcasts(**model, show_train=False)
        if not "important" in kwargs:
            try:
                _ = plot_feature_importance(**model)
            except:
                print("no feature importance information")
    
    return model, print_scores(**model)

#### Dummy Modell
Lediglich Mittelwert verwenden

In [712]:
from sklearn.dummy import DummyRegressor

dummy = {
    "model_name": "Dummy Regressor",
    "model"     : DummyRegressor(strategy="mean"),
    "splitter"  : sss_split,
    "X"         : X,
    "Y"         : {criterion : Y[[column for column in Y.columns if criterion in column]] for criterion in y_columns}
}

In [ ]:
dummy, scores = evaluate_model(dummy, plot=True)
del(dummy)

#### Naives Modell
Letzer Wert der Station verwenden, welches vorhergesagt werden soll.

In [265]:
# prediction: use last values from station
naive = {
    "model_name"    : "Naives Modell",
    "model"         : None,
    "custom_y_hat"  : {criterion : df[criterion].loc[Y.index] for criterion in y_columns},
    "splitter"      : sss_split,
    "X"             : X,
    "Y"             : {criterion : Y[[column for column in Y.columns if criterion in column]] for criterion in y_columns}
}

In [ ]:
naive, scores = evaluate_model(naive, plot=True)
del(naive)

#### Lineare Regression

##### mit L2 Norm (Ridge)

In [326]:
lin_ridge = {
    "model_name"    : "Linear Ridge",
    "model"         : RidgeCV(alphas = [10**x for x in range(-1,6,3)], scoring="neg_mean_squared_error", fit_intercept=False),
    "splitter"      : sss_split,
    #"important"     : important,
    "X"             : X , #X[[col for col in X.columns if "Quinten" not in col]],
    "Y"             : {criterion : Y[[column for column in Y.columns if criterion in column]] for criterion in y_columns}
}

In [ ]:
lin_ridge, scores = evaluate_model(lin_ridge, plot=True)
save_to_cache(lin_ridge, id="lin_ridge", folder="models")
del(lin_ridge)

##### mit L1 Norm (Lasso)

In [781]:
lin_lasso = {
    "model_name"    : "Linear Lasso",
    "model"         : LassoCV(alphas = [10**x for x in range(-1,6,3)], fit_intercept=False, max_iter=10000, n_jobs=-1), #alphas can be ignored if more cpu power available
    "splitter"      : sss_split,
    #"important"     : important,
    "X"             : X, #X[[col for col in X.columns if "Quinten" not in col]],
    "Y"             : {criterion : Y[[column for column in Y.columns if criterion in column]] for criterion in y_columns}
}

In [ ]:
lin_lasso, scores = evaluate_model(lin_lasso, plot=True)
save_to_cache(lin_lasso, id="lin_lasso", folder="models")
del(lin_lasso)

#### k Nächste Nachbarn

Mittels ExtraTreesRegressor die wichtigsten Merkmale evaluieren, um die Dimensionalität zu reduzieren

In [479]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel
fs_etr = SelectFromModel(ExtraTreesRegressor(n_estimators=50, max_depth=5, max_features=0.1, n_jobs=-1))

In [ ]:
important = {}
df_importances = pd.DataFrame()
for column in tqdm(Y.columns):
    groups = get_y_groups(y.loc[X.index], column.rsplit("_",1)[0])
    # train on a stratified dataset
    for train, test in list(sss_split.split(X, groups)):
        fs_etr.fit(X.iloc[train,:],Y[column].iloc[train])
        important[column] = fs_etr.get_feature_names_out()
        break #once is enough
    df_importances[column] = fs_etr.estimator_.feature_importances_
df_importances.set_index(X.columns, inplace=True, drop=True)

some merging and sorting

In [481]:
backup = df_importances.copy()

In [482]:
df_importances = df_importances.melt(ignore_index=False).reset_index()
df_importances["criterion"] = df_importances["variable"].apply(lambda x: x.split("_")[-2])
df_importances["forcast"] = df_importances["variable"].apply(lambda x: x.split("_")[-1].replace("min","")).astype(int)
df_importances = df_importances.sort_values(["criterion", "forcast", "value"], ascending=[True,True,True])
df_importances["forcast"] = df_importances["forcast"].astype(str)

visualize

In [483]:
title = "Merkmalswichtigkeit bestimmt mit extrem randomisierte Bäume (Extra DT)"
fig = px.bar(df_importances[df_importances["value"]>0.01], y="value", color="forcast", x="index", facet_col="criterion", title=title,
             labels={"index":"", "value":"feature importance"})

fig.update_yaxes(matches=None)
fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))

fig.update_xaxes(matches=None)
fig.for_each_xaxis(lambda xaxis: xaxis.update(showticklabels=True))


# viewing the plot in browser:
if plot_in_browser:
    plot(fig)
if plot_in_notebook:
    fig.show()

# save plot to export
if save_to_folder:
    fig.write_html(export + title + ".html")

In [484]:
kNN = {
    "model_name"    : "kNN151 mit ExtraTree",
    "model"         : KNeighborsRegressor(n_neighbors=151, p=2, leaf_size=100, weights="uniform", n_jobs=-1), #alphas can be ignored if more cpu power available
    "splitter"      : sss_split,
    "important"     : important,
    "X"             : X,
    "Y"             : {criterion : Y[[column for column in Y.columns if criterion in column]] for criterion in y_columns}
}

In [ ]:
kNN, scores = evaluate_model(kNN, plot=True)
save_to_cache(kNN, id="kNN", folder="models")
del(kNN)

#### Entscheidungsbäume

In [784]:
dt = {
    "model_name"    : "DT7_pruning",
    "model"         : DecisionTreeRegressor(criterion="squared_error", ccp_alpha=0.01, max_depth=7),
    "splitter"      : sss_split,
    "X"             : X, #X[[col for col in X.columns if "Quinten" not in col]],
    "Y"             : {criterion : Y[[column for column in Y.columns if criterion in column]] for criterion in y_columns}
}

In [ ]:
dt, scores = evaluate_model(dt, plot=True)
save_to_cache(dt, id="dt", folder="models")
del(dt)

##### Random Forest

In [506]:
from sklearn.ensemble import RandomForestRegressor

rf = {
    "model_name"    : "RF_30DT7",
    "model"         : RandomForestRegressor(n_estimators=30, criterion="squared_error", max_depth=7, ccp_alpha=0.01, n_jobs=-1, bootstrap=True, max_samples=.1),
    "splitter"      : sss_split,
    "X"             : X,
    "Y"             : {criterion : Y[[column for column in Y.columns if criterion in column]] for criterion in y_columns}
}

In [ ]:
rf, scores = evaluate_model(rf, plot=True)
save_to_cache(rf, id="rf", folder="models")
del(rf)

#### Neuronales Netz

In [786]:
mlp = {
    "model_name"    : "MLP 5",
    "model"         : MLPRegressor(hidden_layer_sizes=(5), shuffle=True, early_stopping=True, learning_rate="adaptive", max_iter=400),
    "splitter"      : sss_split,
    "X"             : X,
    "Y"             : {criterion : Y[[column for column in Y.columns if criterion in column]] for criterion in y_columns}
}

In [ ]:
mlp, scores = evaluate_model(mlp, plot=True)
save_to_cache(mlp, id="mlp", folder="models")
del(mlp)

In [788]:
mlp32 = {
    "model_name"    : "MLP 32 optimistic",
    "model"         : MLPRegressor(hidden_layer_sizes=(32), shuffle=True, early_stopping=True, learning_rate="adaptive", max_iter=400),
    "splitter"      : sss_split,
    "X"             : X,
    "Y"             : {criterion : Y[[column for column in Y.columns if criterion in column]] for criterion in y_columns}
}

In [ ]:
mlp32, scores = evaluate_model(mlp32, plot=True)
save_to_cache(mlp32, id="mlp_opt", folder="models")
del(mlp32)

### Kombination der Modelle

In [534]:
from sklearn.ensemble import VotingRegressor, BaggingRegressor

In [794]:
voting = VotingRegressor(
    estimators=[
        ('lin', RidgeCV(alphas = [10**x for x in range(-1,6,3)], scoring="neg_mean_squared_error", fit_intercept=False)), 
        ('knn_etr', make_pipeline(SelectFromModel(ExtraTreesRegressor(n_estimators=50, max_depth=5, max_features=0.1, n_jobs=-1)),
                                  KNeighborsRegressor(n_neighbors=151, p=2, leaf_size=100, weights="uniform", n_jobs=-1))),
        ('dt', DecisionTreeRegressor(criterion="squared_error", ccp_alpha=0.01, max_depth=7)),
        ('rf', RandomForestRegressor(n_estimators=30, criterion="squared_error", max_depth=7, ccp_alpha=0.01, n_jobs=-1, bootstrap=True, max_samples=.1)),
        ('mlp', MLPRegressor(hidden_layer_sizes=(5), shuffle=True, early_stopping=True, learning_rate="adaptive", max_iter=400))
        ]
    )
voting

VotingRegressor(estimators=[('lin',
                             RidgeCV(alphas=[0.1, 100, 100000],
                                     fit_intercept=False,
                                     scoring='neg_mean_squared_error')),
                            ('knn_etr',
                             Pipeline(steps=[('selectfrommodel',
                                              SelectFromModel(estimator=ExtraTreesRegressor(max_depth=5,
                                                                                            max_features=0.1,
                                                                                            n_estimators=50,
                                                                                            n_jobs=-1))),
                                             ('kneighborsregressor',
                                              KNeighborsRegressor(leaf_size=100,
                                                                  n_jobs=-1,
                                                                  n_neighbors=151))])),
                            ('dt',
                             DecisionTreeRegressor(ccp_alpha=0.01,
                                                   max_depth=7)),
                            ('rf',
                             RandomForestRegressor(ccp_alpha=0.01, max_depth=7,
                                                   max_samples=0.1,
                                                   n_estimators=30,
                                                   n_jobs=-1)),
                            ('mlp',
                             MLPRegressor(early_stopping=True,
                                          hidden_layer_sizes=5,
                                          learning_rate='adaptive',
                                          max_iter=400))])

In [795]:
voting = {
    "model_name"    : "Voting",
    "model"         : voting
    "splitter"      : sss_split,
    "X"             : X,
    "Y"             : {criterion : Y[[column for column in Y.columns if criterion in column]] for criterion in y_columns}
}

In [ ]:
voting, scores = evaluate_model(voting, plot=True)
save_to_cache(voting, id="combi", folder="models")
del(voting)

## Modellvergleich

In [881]:
def plot_score_of_all_models(all_models):
    all_scores = pd.DataFrame()
    for modelname, score_types in all_models.items():
        scores = pd.concat([score_types["scores"], score_types["test_scores"]]).drop(columns=["submodels","fold","train_size"]).melt(id_vars=["source", "criterion", "forcast time"], 
                                                                                                                                     value_name="score_value", var_name="score_name")
        scores["model_name"] = modelname
        all_scores = pd.concat([all_scores, scores])
    
    #all_scores = all_scores[all_scores["score_name"].isin(["mean_absolute_error", "root_mean_squared_error"])]
    
    fig1 = px.scatter(all_scores[all_scores["source"]=="testing"], facet_col="score_name", facet_row="criterion", x="forcast time", y="score_value", 
                      color="model_name", title="Metriken der entwickelten Modelle" + "<br><sub>X -> Testing", 
                      labels={
                          "score_value":"Fehler [km/h]",
                          "model_name": "Modell",
                          "forcast time":"Vorhersagezeit"
                      })

    fig1.update_traces(marker=dict(size=5, symbol="x"), showlegend=False)
    fig1.update_yaxes(matches=None)
    fig1.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig1.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
    #add competitors
    fig1.add_hline(y=8.66, line_dash="solid", row=0, col=1, line_color="red", line_width=3,
        annotation_text="MF-AROME", 
        annotation_position="top left")
    fig1.add_hline(y=12.01, line_dash="solid", row=0, col=2, line_color="red", line_width=3,
        annotation_text="MF-AROME", 
        annotation_position="top left")
    fig1.add_hline(y=0.14, line_dash="solid", row=0, col=3, line_color="red", line_width=3,
        annotation_text="MF-AROME", 
        annotation_position="bottom left")
    
    fig1.add_hline(y=4.96, line_dash="solid", row=1, col=1, line_color="red", line_width=3,
        annotation_text="MM-SWISS1K", 
        annotation_position="top left")
    fig1.add_hline(y=6.67, line_dash="solid", row=1, col=2, line_color="red", line_width=3,
        annotation_text="MM-SWISS1K", 
        annotation_position="top left")
    fig1.add_hline(y=-1.29, line_dash="solid", row=1, col=3, line_color="red", line_width=3,
        annotation_text="MM-SWISS1K", 
        annotation_position="bottom left")

    fig2 = px.box(all_scores[all_scores["source"]=="training"], facet_col="score_name", facet_row="criterion", x="forcast time", y="score_value", color="model_name")
    fig = go.Figure(data=fig1.data + fig2.data, layout = fig1.layout)


    # viewing the plot in browser:
    if plot_in_browser:
        plot(fig)
    if plot_in_notebook:
        fig.show()

    # save plot to export
    if save_to_folder:
        fig.write_html(export + "Metriken der entwickelten Modelle.html")
    return fig, all_scores

In [882]:
_, all_model_scores = plot_score_of_all_models(all_models)

In [892]:
all_model_scores.to_csv(PROJECT_DIR + "src/models/all_model_scores.zip")

save to cache


True